In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import models

plt.rcParams['figure.figsize']=[16,8]

In [ ]:
def str2numpy(src):
    # convert list of string to numpy array
    return np.asarray([s.split() for s in src], dtype=int)

def normalize(src, axis=0):
    mean = src.mean(axis, keepdims=True)
    std = src.std(axis, keepdims=True)
    dst = (src - mean) / (std + 1e-9)
    return dst

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = models.Classifier()
net = net.to(device)
net.load_state_dict(torch.load('logs/model.pth'))

In [ ]:
df = pd.read_csv('../signal_process/sample/fsr.csv')
data = str2numpy(df['sensor'])

In [ ]:
i0 = 5000
size = 1024
s = normalize(data[i0:i0+size])
plt.plot(s.sum(1))

In [ ]:
st = torch.FloatTensor(s.T).unsqueeze(1)
with torch.no_grad():
    out = net(st)
    reliability = out.squeeze().numpy()
reliability_mean = reliability.mean(1)

plt.figure(figsize=(16, 24))
for i in range(32):
    plt.subplot(16, 4, i+1)
    plt.plot(s[:, i])
    plt.title('%f' % reliability_mean[i])